In [ ]:
#We are repeating the SVR experiment with the same data but splitting the test and train data manually to ensure we only use the suggested data as test data.

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

#load the data
filepath = r"C:\Users\Admin\Machine Learning Fundamentals\CSV_files\MLData.xlsx"
df = pd.read_excel(filepath)
